In [23]:
%load_ext blackcellmagic
import pandas as pd
import numpy as np
from itertools import chain
from math import floor

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [44]:
total_csvs = !ls total_*

In [45]:
def write_excel(file_name: str, dfs, sheets):
    writer = pd.ExcelWriter(f'{file_name}.xlsx')
    [df.to_excel(writer, sheet) for df, sheet in zip(dfs, sheets)]
    writer.save()


def read_csv(file_name: str, **args: dict) -> pd.DataFrame:
    return pd.read_csv(filepath_or_buffer=file_name, **args)


def file_name_suffix(name: str) -> str:
    return name[name.rfind('_') + 1:-4]


total_names = ['fuel', 'co2']
df_totals = {
    file_name_suffix(name): read_csv(
        file_name=name, encoding='utf-8', names=total_names, header=0)
    for name in total_csvs
}
df_new_co2 = read_csv(
    file_name='fordonspark.csv',
    encoding='utf-8',
    header=0,
    names=['license_nbr', 'co2'])
df_cars = pd.read_csv(
    'cars.csv',
    encoding='latin1',
    sep=';',
    header=0,
    names=[
        'license_nbr', 'brand', 'year', 'driver', 'region', 'consumption',
        'co2', 'fuel'
    ])
df_cars.head(20)

,license_nbr,brand,year,driver,region,consumption,co2,fuel
0,YZF 336,VW TRANSPORTER,2018,ALBIN LÖFSTRAND,ÖST,"0,62",161,Diesel
1,PYN 113,MB SPRINTER,2018,ALEXANDER WERNERSSON,SYD,"0,81",213,Diesel
2,JUB 435,VW TRANSPORTER SKÅP T6,2016,ANDERS ANDERSSON,SYD,"0,77",203,Diesel
3,YLJ 847,MB SPRINTER 316 CDI SKÅP,2017,ANDERS BONNIER,SYD,"0,81",213,Diesel
4,YHN 351,AUDI Q5 2.0 QUATTRO,2017,ANDERS KARLSSON,ÖST,"0,59",154,Diesel
5,BAP 210,VW CADDY MAXI SKÅP,2017,ANDERS NILSSON,SYD,"0,50",132,Diesel
6,RSA 761,VW TRANSPORT CRAFTER SKÅP 35,2015,ANDREAS ENGSTRÖM,ÖST,"0,84",221,Diesel
7,YHR 669,BMW 320D XDRIVE,2016,ANDREAS FRITZON,MITT,"0,47",124,Diesel
8,EFM 275,BMW 320D SEDAN BUSINESS ADVATAGE EDITION,2017,ANDREAS KNUTSEN-ÖY,MITT,"0,44",116,Diesel
9,ELW 214,VW TRANSPORT CRAFTER 37,2016,ANDREAS LÖÖV,MITT,"0,84",221,Diesel


In [46]:
df_cars["brand"] = df_cars["brand"].apply(lambda b: " ".join(b.upper().split(" ")[:2]))
df_cars["co2"] = df_new_co2["co2"]
df_cars.head()


,license_nbr,brand,year,driver,region,consumption,co2,fuel
0,YZF 336,VW TRANSPORTER,2018,ALBIN LÖFSTRAND,ÖST,"0,62",36464,Diesel
1,PYN 113,MB SPRINTER,2018,ALEXANDER WERNERSSON,SYD,"0,81",45974,Diesel
2,JUB 435,VW TRANSPORTER,2016,ANDERS ANDERSSON,SYD,"0,77",44114,Diesel
3,YLJ 847,MB SPRINTER,2017,ANDERS BONNIER,SYD,"0,81",45974,Diesel
4,YHN 351,AUDI Q5,2017,ANDERS KARLSSON,ÖST,"0,59",18098,Diesel


In [47]:
df_totals['work']

,fuel,co2
0,Konventionell diesel,16365
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),14384
2,Konventionell bensin,15930
3,Bensin E4.8 (4.8% bioinblandning),15875
4,Etanol (E85),16836
5,Fordonsgas,10126
6,Elfordon. 39 kWh,14311
7,Elfordon. 17 kWh,10358
8,Elfordon. 100 kWh,25367
9,Laddhybrid,12188


In [48]:
df_totals['big']

,fuel,co2
0,Konventionell diesel,45530
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),37334
2,Gasfordon,26009


In [49]:
df_totals['medium']

,fuel,co2
0,Konventionell diesel,36418
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),32744
2,Konventionell bensin,37787
3,Biobensin E4.8 (4.8% bioinblandning),34988


In [50]:
df_totals['small']

,fuel,co2
0,Konventionell diesel,27742
1,Biodiesel B25.5 (25% inblandning av FAME/HVO),23182
2,Konventionell bensin,32822
3,Biobensin E4.8 (4.8% bioinblandning),29587
4,Elfordon skåp. 26.7 kWh,13543
5,Gasfordon,16951


In [51]:
car_cats = {
    "service": {
        "small": ["VW CADDY"],
        "medium": ["MB VITO", "VW TRANSPORTER"],
        "big": ["MB SPRINTER", "VW TRANSPORT"],
    },
    "work": [
        "AUDI A6",
        "AUDI Q5",
        "BMW 220D",
        "BMW 318D",
        "BMW 320D",
        "SKODA SUPERB",
        "VOLVO S60",
        "VOLVO S90",
        "VOLVO V60",
        "VOLVO V90",
        "VOLVO XC40",
        "VOLVO XC60",
        "VOLVO XC70",
        "VW PASSAT",
        "VW TIGUAN",
        "VW TOUAREG",
    ],
}

service_brands = set(chain(*car_cats["service"].values()))
work_brands = set(car_cats["work"])


In [52]:
BIODIESEL = "Biodiesel B25.5 (25% inblandning av FAME/HVO)"
DIESEL = "Konventionell diesel"
BENSIN = "Konventionell bensin"

def scen_1a(totals, df_cars):
    df = df_cars.query(
        "(brand in @service_brands & year < 2015) | (brand in @work_brands & year < 2017)"
    )
    return totals, df


# Remove biodiesel from all
def scen_1b(totals, df_cars):
    d = {}
    _, df_cars = scen_1a(totals, df_cars)
    for cat, df in totals.items():
        dropped = df.drop(df[df["fuel"] == BIODIESEL].index)
        d[cat] = dropped
    return d, df_cars


def scen_1c(totals, df_cars):
    # Samma som scenario 1, men räkna med svensk elmix.
    pass


def scen_1d(totals, df_cars):
    # Samma som scenario 1, men räkna med europeisk elmix.
    pass


def scen_2(totals, df_cars):
    # Byt färre fordon, de som vars leasingperiod har utgått, sätt mängden utbytta fordon till 10% => 28 av 282.
    pass


def scen3(totals, df_cars):
    d = {}
    for cat, df in totals.items():
        rng = df[(df['fuel'] != BENSIN) & (df['fuel'] != DIESEL)]
        dropped = df.drop(rng.index)
        d[cat] = dropped
    return d, df_cars


def scen4(totals, df_cars):
    # Byt samtliga fordon, även om leasingperioden inte utgått, förutsatt att det finns alternativ som har lägre utsläppsvärden. Därmed ej tillämpbart på företaget men ger en mer allmän teoretisk tillämpning.
    pass


def scen5(totals, df_cars):
    pass


# Find best fuel value for every category
def find_opt_co2(get_scen, totals, df_cars):
    cat_co2_dict = {}
    co2_fuel_dict = {}
    d, df_cars = get_scen(totals, df_cars)
    for cat, df in d.items():
        min_idx = df["co2"].idxmin()
        fuel, co2 = df.loc[min_idx].values
        cat_co2_dict[cat] = co2
        co2_fuel_dict[co2] = fuel
    return df_cars, cat_co2_dict, co2_fuel_dict


In [66]:
def get_car_cat(brand):
    if brand in work_brands:
        return "work"
    for k, v in car_cats["service"].items():
        if brand in set(v):
            return k
    raise Exception(f"Could not classify brand {brand}")


def optimize_big(df, cat_co2_dict):
    # Only replace if medium is better than big
    if cat_co2_dict["big"] < cat_co2_dict["medium"]:
        return df, pd.DataFrame()
    candidates = df.query(
        "category == 'big' & (region == 'ÖST' | region == 'SYD' | region == 'VÄST')"
    ).sort_values("co2", ascending=False)
    candidates = candidates.loc[: floor(len(candidates) * 0.25)]
    df_replaced = df.copy()
    df_replaced.loc[candidates.index, "category"] = "medium"
    return df_replaced, candidates


def find_best_fuels(df, cat_co2_dict, co2_fuel_dict):
    df = df.assign(category=[get_car_cat(brand) for brand in df["brand"]])
    df, big_replaced = optimize_big(df, cat_co2_dict)
    df = df.assign(new_co2=[cat_co2_dict[cat] for cat in df["category"]])
    df.loc[big_replaced.index, "category"] = "big"  # Show the 25% cars as big again
    df = df.assign(new_fuel=[co2_fuel_dict.get(co2, np.nan) for co2 in df["new_co2"]])
    return df


def run_scenarios(scenarios, df_cars):
    dfs, sheets = [], []
    for scen in scenarios:
        df_final = find_best_fuels(*find_opt_co2(scen, df_totals, df_cars))
        df = df_cars.combine_first(df_final)
        df = df.assign(category=[get_car_cat(brand) for brand in df["brand"]])
        dfs.append(df)
        sheets.append(scen.__name__)
    return dfs, sheets


scenarios = [scen_1a, scen_1b, scen3]
dfs, sheets = run_scenarios(scenarios, df_cars)
write_excel("isak", dfs, sheets)
display(dfs[0])

,brand,category,co2,consumption,driver,fuel,license_nbr,new_co2,new_fuel,region,year
0,VW TRANSPORTER,medium,36464,"0,62",ALBIN LÖFSTRAND,Diesel,YZF 336,NaN,NaN,ÖST,2018
1,MB SPRINTER,big,45974,"0,81",ALEXANDER WERNERSSON,Diesel,PYN 113,NaN,NaN,SYD,2018
2,VW TRANSPORTER,medium,44114,"0,77",ANDERS ANDERSSON,Diesel,JUB 435,NaN,NaN,SYD,2016
3,MB SPRINTER,big,45974,"0,81",ANDERS BONNIER,Diesel,YLJ 847,NaN,NaN,SYD,2017
4,AUDI Q5,work,18098,"0,59",ANDERS KARLSSON,Diesel,YHN 351,NaN,NaN,ÖST,2017
5,VW CADDY,small,31034,"0,50",ANDERS NILSSON,Diesel,BAP 210,NaN,NaN,SYD,2017
6,VW TRANSPORT,big,47444,"0,84",ANDREAS ENGSTRÖM,Diesel,RSA 761,NaN,NaN,ÖST,2015
7,BMW 320D,work,15866,"0,47",ANDREAS FRITZON,Diesel,YHR 669,10126.0,Fordonsgas,MITT,2016
8,BMW 320D,work,15278,"0,44",ANDREAS KNUTSEN-ÖY,Diesel,EFM 275,NaN,NaN,MITT,2017
9,VW TRANSPORT,big,47444,"0,84",ANDREAS LÖÖV,Diesel,ELW 214,NaN,NaN,MITT,2016
